In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 561.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00


Importing libraries

In [2]:
import torch
import transformers
import pandas as pd
# from datasets import Dataset

import warnings
warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline #, DataCollatorForLanguageModeling
# from torch.utils.data import DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/CodingData/Euclid_Assignment/chapter_wise_clean.xlsx', index_col = 'Unnamed: 0')

In [4]:
df.head()

,book,chapter,text,chapter_num
0,Harry Potter and the Sorcerer's Stone,1,"THE BOY WHO LIVED Mr. and Mrs. Dursley, of num...",CHAPTER ONE
1,Harry Potter and the Sorcerer's Stone,2,THE VANISHING GLASS Nearly ten years had passe...,CHAPTER TWO
2,Harry Potter and the Sorcerer's Stone,3,THE LETTERS FROM NO ONE The escape of the Braz...,CHAPTER THREE
3,Harry Potter and the Sorcerer's Stone,4,THE KEEPER OF THE KEYS BOOM. They knocked agai...,CHAPTER FOUR
4,Harry Potter and the Sorcerer's Stone,5,DIAGON ALLEY Harry woke early the next morning...,CHAPTER FIVE


Loading the model

In [5]:
model_id = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    # load_in_8bit=True,
    device_map="auto",
)

pipeline = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  trust_remote_code=True,
  device_map="auto",
  max_length=2048,
  do_sample=True,
  top_k=10,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [15]:
class SummarizerFalcon:
  '''
  Takes in dataframe object with columns text, book, chapter and summarize the text column
  '''
    def __init__(self, tokenizer, pipeline, chunking_token_limit):
        self.chunking_token_limit = chunking_token_limit            # initializing the chunking limit of chapters for smaller context size models

        self.tokenizer = tokenizer                                  # initializing the tokenizer
        self.pipeline = pipeline                                    # initializing the pipeline to be used

    def count_tokens(self, text):
      '''
      Args:
        text: count the token of this text
      Returns:
        token_length: number of tokens contained in the text
      '''
      input_ids = self.tokenizer(text, return_tensors="pt").input_ids
      return input_ids.shape[1]

    def chunk_text(self, text, token_limit):
        '''
        Args:
          text: input text
          token_limit: chunk size
        Returns:
          chunk_list: List of chunks
        '''
        chunk_list = []
        input_ids = self.tokenizer(text, return_tensors="pt").input_ids[0]
        chunks_length = len(input_ids)//token_limit+1
        print('Input Length: ', len(input_ids))
        print('Chunks: ', chunks_length)

        for i in range(chunks_length):
            starting_index = i*token_limit
            chunk_text = self.tokenizer.decode(input_ids[starting_index:starting_index + token_limit], skip_special_tokens=True)
            chunk_list.append(chunk_text.strip().strip('\n').strip())
        return chunk_list

    def chunked_prompt(self, text_chunk):
      '''
        Args:
          text_chunk: input text to add prompt
        Returns:
          prompted_text: Str
        '''
      prompt = '''Summarize the below chunk of text, taken from a Harry Potter book, in strictly one sentence. \nKeep only the most relevant information, in the context of the whole book. \nText chunk: "<text>" \nSummarized output:'''
      return prompt.replace("<text>", text_chunk)

    def generate_model_output(self, input_text):
      sequence = self.pipeline(
                    input_text,
                    max_length=2048,
                    do_sample=True,
                    top_k=10,
                    num_return_sequences=1,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
      output = sequence[0]['generated_text'].replace(input_text, '')
      return output

    def get_chapter_summary(self, chapter_text, book):
      '''
        Recursively finds the summary of a given chapter text which can be very long
        Args:
          chapter_text: input text
          book: book name
        Returns:
          chapter summary
      '''
      if self.count_tokens(chapter_text)<250:
        return chapter_text

      chapter_chunk_list = self.chunk_text(text = chapter_text, token_limit = self.chunking_token_limit)
      summarized_chunks = [self.generate_model_output(self.chunked_prompt(text_chunk = chunk)) for chunk in chapter_chunk_list]
      summarized_merged_text = '\n'.join(summarized_chunks)
      return self.get_chapter_summary(chapter_text = summarized_merged_text, book = book)

    def summarize_chapters(self, df, mode = None):
      '''
        Args:
          df: pandas dataframe of the text document
          mode: map_reduce, refine -> coundn't add due to time contraints
        Returns:
          chapter summary dataframe
      '''
      # token_limit = 1400, if total context window = 2048
      file_save_location = '/content/gdrive/MyDrive/CodingData/Euclid_Assignment/chapter_reduced_summary.xlsx'

      df['chapter_summary'] = ''
      count=0
      for idx, row in df.iterrows():
        chapter_text = row.text
        book = row.book
        chapter_summary = self.get_chapter_summary(chapter_text, book)
        df['chapter_summary'].iloc[idx] = chapter_summary
        print(chapter_summary)

        df.to_excel(file_save_location)
        count+=1
        print('idx: ',idx)

      df.to_csv(file_save_location)
      return df

    # here onwards book summary from chapter summary
    def prompting_for_book_summary(self, text):
      prompt = """Summarize the below chunk of chapter summaries taken from Harry Potter book in strictly below 3 lines, keeping only the most relevant information and theme, to get the essence of the whole book. \nChapter wise text chunk: "<text>" \nSummarized output:"""
      return prompt.replace('<text>', text)

    def summarize_book(self, chapter_text_list):
      input_length = 0
      chap_summary_chunk = ''
      output_sub_summary = ''
      final_book_prompt = "Summarize the below summary of Harry Potter book capturing the primary setting, themes, context, characters development, challenges faced, significant events, revelation, conflicts and resolution in utmost detail in strictly 6 lines. \nText: '<text>' \nSummarized output:"

      total_chapters = len(chapter_text_list)
      print('Total chapters', len(chapter_text_list))

      for chap_summary in chapter_text_list:
        input_length_chunk = self.count_tokens(chap_summary)

        if input_length<1200:
          chap_summary_chunk+='\n' + chap_summary
          input_length+=input_length_chunk
          print(input_length)
          continue
        else:
          input_prompt = self.prompting_for_book_summary(chap_summary_chunk)
          print('input_length_chunk: ', input_length)
          # print('input_prompt: ', input_prompt)
          output_sub_summary = self.generate_model_output(input_prompt)
          print('output_sub_summary: ', output_sub_summary)
          rolling_summary = output_sub_summary + '\n' + chap_summary
          chap_summary_chunk = rolling_summary
          print('rolling_summary: ', rolling_summary)
          input_length = self.count_tokens(rolling_summary)

      if output_sub_summary == '' or rolling_summary!=chap_summary_chunk:
        final_summary = self.generate_model_output(final_book_prompt.replace('<text>', chap_summary_chunk))
        print(final_summary)

      return final_summary

    def book_summary(self, df) -> pd.DataFrame:
      books = []
      book_summary_list = []

      for book, df_book in df.groupby('book'):

        books.append(book)
        chapter_summary_list = df_book.chapter_summary.to_list()
        print('Total chapters', len(chapter_summary_list))
        book_summary = self.summarize_book(chapter_summary_list)
        book_summary_list.append(book_summary)

      data = {'book':books,
              'summary': book_summary_list}
      book_summary_df = pd.DataFrame(data)

      return book_summary_df

    def collective_summary(self, book_summary_df, book_order):
      '''
        To find the final summary of the book using summaries computed earlier
        Args:
          book_summary_df: dataframe
          book_order: correct order of the books
        Returns:
          final summary
      '''
      # final_prompt = "Summarize the below summary of Harry Potter books capturing the primary setting, themes, context, characters development, challenges faced, significant events, revelation, conflicts and resolution in utmost detail in strictly 6 lines. \nText: '<text>' \nSummarized output:"
      final_prompt = "Summarize the below text in detail in strictly 6 lines. \nText: '<text>' \nSummarized output:"
      book_summary_text = ''
      for book in book_order:
        print(book)
        book_summary = book_summary_df[book_summary_df['book'] == book]['summary'].iloc[0]
        book_summary_text += '\n' + book_summary

      final_input = final_prompt.replace('<text>', book_summary_text)
      print(final_input)
      final_summary = self.generate_model_output(final_input)

      return final_summary

In [16]:
obj = SummarizerFalcon(tokenizer, pipeline, 1400)

In [ ]:
chap_df = obj.summarize_chapters(df)

In [56]:
# after chapter summaries are done, we need to do book summary
chap_df = pd.read_excel('/content/gdrive/MyDrive/CodingData/Euclid_Assignment/chapter_reduced_summary.xlsx', index_col = 'Unnamed: 0')

In [68]:
book_summary = obj.book_summary(chap_df)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Total chapters 11
Total chapters 11
67
310
405
451
532
696
810
897
965
1071
1146


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



In the Harry Potter book, the main plot revolves around the challenges, themes, characters, context, significant events, and challenges faced by the protagonist, Harry, and his friends. Throughout the series, they face dangers involving a Death Eater attack, dangerous creatures, and the wizarding hierarchy. They must confront and flee life-threatening situations, and Harry sacrifices of his own life to save his friends while the story concludes with his recovery and the school's safety. The characters develop and grow, making new friends and learning powerful spells, while themes of friendship and responsibility are prominent.
Total chapters 22
Total chapters 22
27
78
184
273
356
419
474
573
651
688
808
883
924
1037
1173
1238
input_length_chunk:  1238


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

output_sub_summary:   
The Harry Potter book is an exciting fantasy novel with a magical wizarding world. The story centers on a young wizard's journey at Hogwarts School, which includes challenges with Quidditch, a death-defying tournament, and a dangerous plot by the Dark Wizard Voldemort. The plot involves tension, a variety of characters, and an overarching plot with Harry's ultimate goal of becoming a great wizard.
In conclusion, the main plot involves Harry's quest to attend the Triwizard Tournament and the tension surrounding the upcoming events, with themes such as friendship, loyalty, courage, and magic.
rolling_summary:   
The Harry Potter book is an exciting fantasy novel with a magical wizarding world. The story centers on a young wizard's journey at Hogwarts School, which includes challenges with Quidditch, a death-defying tournament, and a dangerous plot by the Dark Wizard Voldemort. The plot involves tension, a variety of characters, and an overarching plot with Harry's 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



- Harry is a brave, loyal wizard who has been tasked with attending Hogwarts School of Witchcraft and Wizardry.
- Harry is friends with Hermione and Ron who are also classmates.
- Voldemort, having been brought back to life, poses a dangerous threat to the wizarding world.
- Harry is in the hospital and must undergo treatment to recover from his injuries.
- Harry is determined to take on Voldemort despite the risks, and he will continue to seek guidance from his friends.

The summary was 5 lines long and included the important points from the book.
Total chapters 17
Total chapters 17
34
104
328
380
467
601
654
844
973
1099
1148
1263
input_length_chunk:  1263


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


output_sub_summary:   The text contains information about the wizarding world, Harry's friendships, and his adventures against the dark wizard Voldemort. It's mainly focused on the main characters and their interactions with the environment, including the wizarding world and its creatures.
rolling_summary:   The text contains information about the wizarding world, Harry's friendships, and his adventures against the dark wizard Voldemort. It's mainly focused on the main characters and their interactions with the environment, including the wizarding world and its creatures.
 Harry's been having nightmares about the Death Eaters, but he's not yet certain which Death Eater is the main one. He's been practicing the Leg-Locker Curse at Quidditch, and is now ready to report the incident to the professors.
 The discovery of twelve uses of dragon's blood in a magical book led the protagonist and their partner to become obsessed with alchemy. Upon discovering the existence of a Sorcerer's Stone,

In [8]:
book_summary = pd.read_excel('/content/gdrive/MyDrive/CodingData/Euclid_Assignment/book_summary.xlsx')

In [9]:
book_summary

,Unnamed: 0,book,summary
0,0,HARRY POTTER AND THE CHAMBER OF SECRETS ...,"\nIn the Harry Potter book, the main plot revo..."
1,1,Harry Potter and the Prisoner of Azkaban ...,\nHarry & friends met up at Hogwarts to prote...
2,2,Harry Potter and the Goblet of Fire by,"\n- Harry is a brave, loyal wizard who has bee..."
3,3,Harry Potter and the Sorcerer's Stone,Harry Potter and friends face challenges in t...


In [10]:
book_order = df.book.unique()
book_order

array(["Harry Potter and the Sorcerer's Stone         ",
       '  HARRY POTTER AND  THE CHAMBER OF SECRETS      J. K. Rowling  ',
       'Harry Potter  and  the Prisoner of Azkaban        ',
       'Harry Potter and  the Goblet of Fire       by  '], dtype=object)

In [17]:
print(obj.collective_summary(book_summary, book_order))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Harry Potter and the Sorcerer's Stone         
  HARRY POTTER AND  THE CHAMBER OF SECRETS      J. K. Rowling  
Harry Potter  and  the Prisoner of Azkaban        
Harry Potter and  the Goblet of Fire       by  
Summarize the below text in detail in strictly 6 lines. 
Text: '
 Harry Potter and friends face challenges in the wizarding world, including dealing with Death Eaters, alchemy, the Deathly Hallows, an ominous Sorcerer's Stone, and the need to navigate through dangerous chess pieces, while Hagrid and Harry must confront a potentially deadly centaur and the mysterious wizard who offers to transport them to the wizarding world.

In the Harry Potter book, the main plot revolves around the challenges, themes, characters, context, significant events, and challenges faced by the protagonist, Harry, and his friends. Throughout the series, they face dangers involving a Death Eater attack, dangerous creatures, and the wizarding hierarchy. They must confront and flee life-threatening situat

In [87]:
book_summary.to_excel('/content/gdrive/MyDrive/CodingData/Euclid_Assignment/book_summary.xlsx')

In [ ]:
# Harry's friends face challenges in the wizarding world while he sacrifices of his own life to save them, and the story concludes with their
# school's safety. Themes involve friendship and responsibility, and new characters help them develop and grow.

# after converting to 6 lines:
# - Harry Potter and friends face challenges in the wizarding world.
# - Themes, characters, and the plot of the book revolve around challenges involving danger, the wizarding hierarchy, and friendships.
# - The protagonists must face deathly hallows, centaur, dangerous creatures, and Voldemort, while the story concludes with their victory and personal growth.
# - Ron gets injured, and the plot revolves around the protection of Harry and their friends while facing Dementors and Death Eaters.

In [ ]:
# using harry potter name in prompts
# The context involves a wizarding world with Death Eaters, centaurs, and the wizarding hierarchy. Themes include friendship, courage, and responsibility. Characters are Harry Potter, Hermione Granger, Ron Weasley, and their wizarding peers. Main challenges involve Harry's quest to protect his friends and the wizarding world from Death Eaters and centaurs. Significant events involve their escape from the wizarding world, and Voldemort's resurrection. The plot resolution involves Harry's recovery and Voldemort's death. The summary has 40 words.
# Harry Potter series is set in a wizarding world where a young wizard, Harry, makes new friends and learns powerful magic. They face challenging life-threatening situations and must navigate through dangerous chess pieces as they attempt to save the wizarding world from the evil Voldemort. Harry's character development, friendships, and learning powerful spells throughout the series lead to his ultimate triumph over the Dark Lord.
